In [1]:
# for webscraping
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time

from bs4 import BeautifulSoup

import numpy as np  # for multi-dimensional containers 
import pandas as pd # for DataFrames
import re # regex

In [2]:
def xpath_wait(x_path,time):
    '''
    handles wait for loading elements found by xpath
    '''
    wait = WebDriverWait(driver, time)
    return wait.until(EC.element_to_be_clickable((By.XPATH,
            x_path)))

def class_wait(class_name,time):
    '''
    handles wait for loading elements found by class name
    '''
    wait = WebDriverWait(driver, time)
    return wait.until(EC.element_to_be_clickable((By.CLASS_NAME,
            class_name)))

In [3]:
# set up driver arguements
option = webdriver.ChromeOptions()
option.add_argument('--no-sandbox')
option.add_argument("--disable-setuid-sandbox")
option.add_argument("--start-maximized")

In [12]:
profiles = ['https://www.instagram.com/barackobama/','https://www.instagram.com/berniesanders/',
            'https://www.instagram.com/joebiden/','https://www.instagram.com/elizabethwarren/',
           'https://www.instagram.com/sentedcruz/','https://www.instagram.com/mikepence/',
            'https://www.instagram.com/realdonaldtrump/']

In [15]:
# columns = ['name','post_url','post_type', 'caption', 'comments','mdy_time','datetime','views','likes']
# df1 = pd.DataFrame(columns=columns)
# df1
df1 = pd.read_csv('politicians_ig.csv')
df1

,name,post_url,post_type,caption,comments,mdy_time,datetime,views,likes
0,Barack Obama,https://www.instagram.com/p/B_7-0jIg5I9/,image,Our medical professionals continue to put thei...,"['Best president ever! Please come back 🤧🥺', '...","May 8, 2020",2020-05-08T18:38:12.000Z,NaN,"796,303"
1,Barack Obama,https://www.instagram.com/p/B_2ViocAHAw/,video,There’s no limit to the devotion I’ve seen in ...,['Theres still room on mt. Rushmore for you Ba...,"May 6, 2020",2020-05-06T14:02:33.000Z,"1,251,554","218,716"
2,Barack Obama,https://www.instagram.com/p/B_z4R5ig29Z/,image,I’ve always loved joining commencements––the c...,['The #Classof2020 deserves to be celebrated f...,"May 5, 2020",2020-05-05T15:07:07.000Z,NaN,"1,572,491"
3,Barack Obama,https://www.instagram.com/p/B_p795VAf35/,video,Just a little over a year ago around Thanksgiv...,"['Best President ever 👏🏆👏 🇧🇷 Loves U', 'You’re...","May 1, 2020",2020-05-01T18:27:43.000Z,"1,749,544","265,540"
4,Barack Obama,https://www.instagram.com/p/B_nIMppAM7b/,image,"Six years ago, I started @MBK_Alliance to addr...","['I MISS YOU 😩', 'Imagine our current Presiden...","Apr 30, 2020",2020-04-30T16:16:05.000Z,NaN,"511,876"
...,...,...,...,...,...,...,...,...,...
1803,President Donald J. Trump,https://www.instagram.com/p/B6U2EvCBb48/,video,"#Repost @whitehouse\n・・・\n""This is a truly his...","['❤️', 'He calls it an investment I call it MA...","Dec 20, 2019",2019-12-21T07:14:40.000Z,"805,163","203,038"
1804,President Donald J. Trump,https://www.instagram.com/p/B6T-xylh25H/,image,THANK YOU! #KAG2020🇺🇸,"['ای ب درد نخور', 'چرا کامنت رو بستی', 'دستم ب...","Dec 20, 2019",2019-12-20T23:11:14.000Z,NaN,"421,932"
1805,President Donald J. Trump,https://www.instagram.com/p/B6REiswhYzR/,image,NaN,"['the people of Iran love you', 'Plz look on m...","Dec 19, 2019",2019-12-19T20:03:53.000Z,NaN,"511,023"
1806,President Donald J. Trump,https://www.instagram.com/p/B6REQi_h-9i/,image,NaN,['خیلی خری ترسو🤣 ایران دوتا پایگاه تونو زد ولی...,"Dec 19, 2019",2019-12-19T20:01:24.000Z,NaN,"829,828"


In [32]:
# first log in to instagram
driver = webdriver.Chrome('C:/Users/tyler/chromedriver_win32/chromedriver.exe', options = option)
driver.get('http://instagram.com')

# fill out elts once clickable
wait = WebDriverWait(driver, 5)
un_field = xpath_wait('//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[2]/div/label/input',10)
un_field.send_keys('angelsofami')
pw_field = xpath_wait('//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[3]/div/label/input',10)
pw_field.send_keys('scrapingforami')
login_btn = xpath_wait('//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[4]/button',10)
login_btn.click()
# close_notif = xpath_wait('/html/body/div[4]/div/div/div[3]/button[2]',10)
# close_notif.click()

In [39]:
# then begin looping through pages
post_count = 215 # number of posts to look at (manually change for each profile)

driver.get(profiles[5])
name = class_wait('rhpdm',20).text
posts = driver.find_elements_by_xpath('//a[contains(@href, "/p/")]')
rows = []
# loop through posts using next arrow
start = 0
for j in range(start,post_count):
    row = []
    print(str(j))
    actions = webdriver.common.action_chains.ActionChains(driver) # allows for clicking at specific coordinates 
    if j == start: # open first post
#         post = posts[0]
        time.sleep(45)
        post = xpath_wait('//a[contains(@href, "/p/B7HKHl0pDJJ/")]',20) # manually scroll to post
    else: # next arrow to go to next
        post = class_wait('coreSpriteRightPaginationArrow',10)
    post.click()

    # get post_url
    post_url = driver.current_url
    
    # get caption
    try: #len(driver.find_elements_by_xpath('/html/body/div[4]/div[2]/div/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span')) > 0:
        caption = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span',10).text
    except:
        caption = 'NA'

    # get [cmt_count] comments
    comment_list = []
    visible_cmts = len(driver.find_elements_by_class_name('Mr508'))
    if visible_cmts > 0 :
        # print all visible comments
        for c in range(visible_cmts):
            comment = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/div[1]/ul/ul['+str(c+1)+']/div/li/div/div[1]/div[2]/span',10).text
            comment_list.append(comment)

    # get post times in mdy and date time format
    MDY_time = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/div[2]/a/time',10).get_attribute('title')
    datetime = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/div[2]/a/time',10).get_attribute('datetime')
    
    # get post type + views/likes
    # video -> use views xpath
    if len(driver.find_elements_by_xpath('/html/body/div[4]/div[2]/div/article/div[2]/section[2]/div/span/span')) > 0:
        post_type = 'video'
        # click views to show likes
        toggle_likes = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/section[2]/div/span',10)
        toggle_likes.click()
        likes = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/section[2]/div/div/div[4]/span',10).text
        actions = webdriver.common.action_chains.ActionChains(driver) # allows for clicking at specific coordinates
        actions.move_to_element_with_offset(toggle_likes, 100, 0).click().perform() # close popup once likes obtained
        views = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/section[2]/div/span/span',10).text
    # image -> use likes xpath
    else:
        post_type = 'image'
        likes = xpath_wait('/html/body/div[4]/div[2]/div/article/div[2]/section[2]/div/div/button/span',10).text
        views = 'NA'
    # append data after each post, in case of crash
    row.append(name)
    row.append(post_url)
    row.append(post_type)
    row.append(caption)
    row.append(comment_list)
    row.append(MDY_time)
    row.append(datetime)
    row.append(views)
    row.append(likes)
    rows.append(row)

200
201
202
203
204
205
206
207
208
209
210
211
212
213
214


In [40]:
# convert to dataframe
df2 = pd.DataFrame(rows,columns=df1.columns)
# df2
df1 = df1.append(df2)

In [41]:
df1

,name,post_url,post_type,caption,comments,mdy_time,datetime,views,likes
0,Barack Obama,https://www.instagram.com/p/B_7-0jIg5I9/,image,Our medical professionals continue to put thei...,"['Best president ever! Please come back 🤧🥺', '...","May 8, 2020",2020-05-08T18:38:12.000Z,NaN,"796,303"
1,Barack Obama,https://www.instagram.com/p/B_2ViocAHAw/,video,There’s no limit to the devotion I’ve seen in ...,['Theres still room on mt. Rushmore for you Ba...,"May 6, 2020",2020-05-06T14:02:33.000Z,"1,251,554","218,716"
2,Barack Obama,https://www.instagram.com/p/B_z4R5ig29Z/,image,I’ve always loved joining commencements––the c...,['The #Classof2020 deserves to be celebrated f...,"May 5, 2020",2020-05-05T15:07:07.000Z,NaN,"1,572,491"
3,Barack Obama,https://www.instagram.com/p/B_p795VAf35/,video,Just a little over a year ago around Thanksgiv...,"['Best President ever 👏🏆👏 🇧🇷 Loves U', 'You’re...","May 1, 2020",2020-05-01T18:27:43.000Z,"1,749,544","265,540"
4,Barack Obama,https://www.instagram.com/p/B_nIMppAM7b/,image,"Six years ago, I started @MBK_Alliance to addr...","['I MISS YOU 😩', 'Imagine our current Presiden...","Apr 30, 2020",2020-04-30T16:16:05.000Z,NaN,"511,876"
...,...,...,...,...,...,...,...,...,...
10,Mike Pence,https://www.instagram.com/p/B5qSV8MpLhL/,image,Hello Michigan! You can feel the energy and su...,"[So beautiful 😙😙😙😍😍👏👏, 🤢🤢🤢🤢🤢🤢🤢, I’m calling bu...","Dec 4, 2019",2019-12-04T18:34:06.000Z,NA,"22,983"
11,Mike Pence,https://www.instagram.com/p/B5p7pYpJmvD/,video,Thanks to President @realDonaldTrump’s leaders...,[I don’t know why these sights won’t register ...,"Dec 4, 2019",2019-12-04T15:16:37.000Z,"31,144","6,939"
12,Mike Pence,https://www.instagram.com/p/B5olq8MJ6GV/,image,NA,[Get rid of “social security” tax please @mike...,"Dec 3, 2019",2019-12-04T02:44:30.000Z,NA,"17,381"
13,Mike Pence,https://www.instagram.com/p/B4nsWFUpIdk/,image,To the men and women of Black Voices for Trump...,[we Iranian people want to boycott these six E...,"Nov 8, 2019",2019-11-08T21:52:11.000Z,NA,"16,440"


In [42]:
df1.to_csv('politicians_ig.csv',index=False)